In [2]:
import pandas as pd
import numpy as np 
import matplotlib as plt 

tweets = pd.read_csv('combined.csv', encoding='utf-8')
tweets = tweets.drop(['tweet_date_created'], axis=1)
tweets = tweets.drop(['sentiment_score'], axis=1)

In [3]:
duplicates = tweets[tweets.duplicated(subset=['tweet_id'], keep=False)]

if not duplicates.empty:
    print(f"Found {len(duplicates)} duplicate tweet ids. Removing duplicates...")
    tweets.drop_duplicates(subset=['tweet_id'], inplace=True)
else:
    print("No duplicate tweet ids found.")

Found 762643 duplicate tweet ids. Removing duplicates...


In [4]:
tweets = tweets.drop(['language'], axis=1)
tweets = tweets.drop(['tweet_id'], axis=1)

In [5]:
# Define a dictionary that maps sentiment labels to numeric values
sentiment_map = {"NEUTRAL": 0, "POSITIVE": 1, "MIXED": 2, "NEGATIVE": 3}

# Map the sentiment labels to their numeric values
tweets['sentiment_values'] = tweets['sentiment'].map(sentiment_map)

In [6]:
tweets = tweets.drop(['sentiment'], axis=1)
tweets.head(5)

,tweet_text,sentiment_values
0,Bayer Leverkusen goalkeeper Bernd Leno will no...,0
1,Gary Speed v Blackburn at St James in 2001/02 ...,0
2,@ChelseaFC Don't make him regret it and start ...,0
3,"@LiverpoolFF @AnfieldEdition He's a liar, made...",3
4,@theesk @Everton Didn't realise Kenwright is d...,0


In [7]:
# Find the indices of rows with Mixed sentiment
mixed_indices = tweets[tweets['sentiment_values'] == 2].index

# Delete the rows with Mixed sentiment
tweets = tweets.drop(mixed_indices)

In [8]:
import pandas as pd

# Shuffle the DataFrame to ensure that the downsampling is random
tweets = tweets.sample(frac=1, random_state=42)

# Count the number of tweets in each sentiment class
counts = tweets['sentiment_values'].value_counts()

# Find the smallest class size
smallest_size = counts.min()

# Downsample each class to the smallest size
positive_tweets = tweets[tweets['sentiment_values'] == 1].sample(n=smallest_size, random_state=42)
negative_tweets = tweets[tweets['sentiment_values'] == 3].sample(n=smallest_size, random_state=42)
neutral_tweets = tweets[tweets['sentiment_values'] == 0].sample(n=smallest_size, random_state=42)

# Concatenate the downsampled DataFrames
tweets = pd.concat([positive_tweets, negative_tweets, neutral_tweets], ignore_index=True)

# Print the new counts of tweets in each class
print('No of positive tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 1])))
print('No of negative tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 3])))
print('No of neutral tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 0])))


No of positive tagged tweets is: 354501
No of negative tagged tweets is: 354501
No of neutral tagged tweets is: 354501


No stopwords

In [9]:
import re
import string 
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    return text

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['tweet_text'].apply(preprocess_text)

tweets =  shuffle(tweets).reset_index(drop=True)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import nltk
import dill

# Split the dataset into input (X) and output (y)
X = tweets['processed_text'].values
y = tweets['sentiment_values'].values

# Split the dataset into 70% training and 30% combined validation and testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary dataset (30% of the entire dataset) into 50% validation and 50% testing
# This results in 15% validation and 15% testing of the entire dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training data size:", len(X_train))
print("Validation data size:", len(X_val))
print("Testing data size:", len(X_test))

Training data size: 744452
Validation data size: 159525
Testing data size: 159526


In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import nltk

text_clf = Pipeline([
    ('vect', CountVectorizer(max_features=5000, ngram_range=(1,1))),
    ('clf', MultinomialNB(alpha=0.01))
])

text_clf.fit(X_train.tolist(), y_train)

y_pred = text_clf.predict(X_val.tolist())
accuracy = accuracy_score(y_val, y_pred)

print("Accuracy on test data: {:.4f}".format(accuracy))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
cr = classification_report(y_val, y_pred)
print("Classification Report:\n", cr)


Accuracy on test data: 0.7376
Confusion Matrix:
 [[30160 10079 12918]
 [ 5936 42094  5006]
 [ 5011  2915 45406]]
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.57      0.64     53157
           1       0.76      0.79      0.78     53036
           3       0.72      0.85      0.78     53332

    accuracy                           0.74    159525
   macro avg       0.74      0.74      0.73    159525
weighted avg       0.74      0.74      0.73    159525



Customized Stopwords

In [23]:
import re
import string 
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from nltk.tokenize import word_tokenize

english_stopwords = stopwords.words('english')

def preprocess_text(text):
    # Get the default NLTK English stopwords
    
    # List of words to keep
    words_to_keep = {"off", "over", "under", "few", "more", "no", "not", "don't", "should", "should've", "aren't", 
                     "couldn't", "didn't", "doesn't", "hadn't", "haven't", "mustn't", "shouldn't", "wasn't", "weren't",
                     "won't", "wouldn't"}

    # Convert english_stopwords to a set, perform the subtraction, and convert the result back to a list
    custom_stopwords = list(set(english_stopwords) - words_to_keep)

    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    # Remove custom stopwords and join the words in a single string
    text = ' '.join([word for word in text.split() if word not in custom_stopwords])
    
    return text

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['tweet_text'].apply(preprocess_text)

tweets =  shuffle(tweets).reset_index(drop=True)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import nltk
import dill

# Split the dataset into input (X) and output (y)
X = tweets['processed_text'].values
y = tweets['sentiment_values'].values

# Split the dataset into 70% training and 30% combined validation and testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary dataset (30% of the entire dataset) into 50% validation and 50% testing
# This results in 15% validation and 15% testing of the entire dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training data size:", len(X_train))
print("Validation data size:", len(X_val))
print("Testing data size:", len(X_test))

Training data size: 744452
Validation data size: 159525
Testing data size: 159526


In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import nltk

text_clf = Pipeline([
    ('vect', CountVectorizer(max_features=5000, ngram_range=(1,1))),
    ('clf', MultinomialNB(alpha=0.01))
])

text_clf.fit(X_train.tolist(), y_train)

y_pred = text_clf.predict(X_val.tolist())
accuracy = accuracy_score(y_val, y_pred)

print("Accuracy on test data: {:.4f}".format(accuracy))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
cr = classification_report(y_val, y_pred)
print("Classification Report:\n", cr)


Accuracy on test data: 0.7371
Confusion Matrix:
 [[29764 11044 12417]
 [ 5933 42561  4604]
 [ 5122  2826 45254]]
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.56      0.63     53225
           1       0.75      0.80      0.78     53098
           3       0.73      0.85      0.78     53202

    accuracy                           0.74    159525
   macro avg       0.74      0.74      0.73    159525
weighted avg       0.74      0.74      0.73    159525



Stopwords

In [36]:
import re
import string 
import pandas as pd
import nltk 
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

# Add custom stopwords
custom_stopwords = ['dont', 'shouldve', 'arent', 'couldnt', 'didnt', 'doesnt', 'hadnt', 'havent', 'mustnt', 'shouldnt', 'wasnt', 'werent', 
                    'wont', 'wouldnt']
english_stopwords.extend(custom_stopwords)

def preprocess_text(text):
    # Get the default NLTK English stopwords
    
    # List of words to keep
    #words_to_keep = {"off", "over", "under", "few", "more", "no", "not", "don't", "should", "should've", "aren't", 
    #                 "couldn't", "didn't", "doesn't", "hadn't", "haven't", "mustn't", "shouldn't", "wasn't", "weren't",
    #                 "won't", "wouldn't"}
    # Create a custom stopwords list
    #custom_stopwords = default_stopwords - words_to_keep
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    # Remove custom stopwords and join the words in a single string
    text = ' '.join([word for word in text.split() if word not in english_stopwords])
    return text

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['tweet_text'].apply(preprocess_text)

tweets =  shuffle(tweets).reset_index(drop=True)


In [37]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import nltk
import dill

# Split the dataset into input (X) and output (y)
X = tweets['processed_text'].values
y = tweets['sentiment_values'].values

# Split the dataset into 70% training and 30% combined validation and testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary dataset (30% of the entire dataset) into 50% validation and 50% testing
# This results in 15% validation and 15% testing of the entire dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training data size:", len(X_train))
print("Validation data size:", len(X_val))
print("Testing data size:", len(X_test))

Training data size: 744452
Validation data size: 159525
Testing data size: 159526


In [48]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import nltk

text_clf = Pipeline([
    ('vect', CountVectorizer(max_features=None, ngram_range=(1,1))),
    ('clf', MultinomialNB(alpha=1.0))
])

text_clf.fit(X_train.tolist(), y_train)

y_pred = text_clf.predict(X_val.tolist())
accuracy = accuracy_score(y_val, y_pred)

print("Accuracy on test data: {:.4f}".format(accuracy))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
cr = classification_report(y_val, y_pred)
print("Classification Report:\n", cr)


Accuracy on test data: 0.7411
Confusion Matrix:
 [[28944 10930 13010]
 [ 5160 42974  5008]
 [ 4075  3123 46301]]
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.55      0.64     52884
           1       0.75      0.81      0.78     53142
           3       0.72      0.87      0.79     53499

    accuracy                           0.74    159525
   macro avg       0.74      0.74      0.73    159525
weighted avg       0.74      0.74      0.73    159525

